# Test notebook for no automated Differential Privacy (DP) data type
## Data Owner
### syft version: 0.6.0

In [1]:
import syft as sy
print(sy.__version__)
import torch
import numpy as np

0.6.0


**DO: login into domain node**  
After launching a domain node with:  
> hagrid launch < node_name > 

(command in wsl)

In [2]:
domain_client = sy.login(
    # url="http://96.20.213.92", → Doesn't work with public IP address right now
    url = "192.168.0.77", # Local IP address
    email="info@openmined.org",
    password="changethis",
    port=8081
)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to 192.168.0.77... done! 	 Logging into free_node... done!


**Create DS user account**

In [3]:
data_scientist_details = {
    "name": "Shu",
    "email": "shu00.song@gmail.com",
    "password": "1234",
    "budget": 9999,
}

domain_client.users.create(**data_scientist_details)

**Upload dataset to domain node**

In [5]:
raw_data = torch.zeros([2,4], dtype=torch.int32)
raw_data[1:] = 1
print(raw_data)

tensor([[0, 0, 0, 0],
        [1, 1, 1, 1]], dtype=torch.int32)


In [6]:
dataset = {"Subject 0": raw_data}
print(dataset)

{'Subject 0': tensor([[0, 0, 0, 0],
        [1, 1, 1, 1]], dtype=torch.int32)}


In [7]:
# We can't use load_dataset() method on not syft.tensor data type (no autoDP)
domain_client.load_dataset(
    name = "Test dataset",
    assets = dataset,
    description = "There's a description") 

Loading dataset... checking asset types...                                                                                                                                    

Exception: ERROR: all private assets must be NumPy ndarray.int32 assets with proper Differential Privacy metadata applied.

Example: syft.Tensor(np.ndarray([1,2,3,4]).astype(np.int32)).private()

or

Example: syft.Tensor([1,2,3,4]).private()

and then follow the wizard. 🧙

In [8]:
# We can use .send() method to upload not autoDP data to domain node
# But we can only load numpy arrays, torch tensors (i.e. not dict)

raw_data.send(domain_client, pointable=True)

<TensorPointer -> free_node:a2e30a3b528b4ff898053bf7f6ea84dd>

In [9]:
dataset.send(domain_client, pointable=True) # Does not work

AttributeError: 'dict' object has no attribute 'send'

In [10]:
# We can check data present on the domain node
domain_client.store.pandas

,ID,Tags,Description,object_type
0,<UID: a2e30a3b528b4ff898053bf7f6ea84dd>,[],,<class 'torch.Tensor'>


**Handle incoming requests (from DS)**  
**- After DS sends a request (go to DS's notebook first)**

In [11]:
domain_client.requests

,Name,Email,Role,Request Type,Status,Reason,Request ID,Requested Object's ID,Requested Object's tags,Requested Budget,Current Budget
0,Shu,shu00.song@gmail.com,Data Scientist,DATA,pending,Please approve!,<UID: f92811eecde041b5b0fae436ccbe3972>,<UID: c76bd8358346445c849b2653b8b0fa54>,[],None,None


In [12]:
domain_client.requests[0].accept()

In [14]:
# Request handler: automatically accept or deny requests
# Doesn't seem to work, even tho it gives no error?
domain_client.requests.add_handler(action="accept")